<a href="https://colab.research.google.com/github/SGRGit/gwOsc/blob/FinalCode/DataPrepForBlipGlitchClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [0]:
import gwpy
import numpy as np
import matplotlib.pyplot as plt
import math

****Analysis with LIGO data for Glitch Recognition using Q-Transforms****

In [0]:
#Import Blip Glitch o1 Data from gspy in Pandas Dataframe
import pandas as pd
raw_data = pd.read_csv("/content/gdrive/My Drive/gspy_blip_o1.csv")

glitch_data = raw_data[['GPStime','ifo', 'peakFreq', 'snr', 'amplitude', 'centralFreq','duration']]

In [0]:
import os
base_dir = "gdrive/My Drive/Colab Notebooks/GlitchNoiseClassification/"
seta = 'Glitch'
setb = 'Noise'

data_dir = os.path.join(base_dir, 'Data')
seta_data_dir = os.path.join(data_dir, seta)
seta_data_fnnames = os.listdir(seta_data_dir)

setb_data_dir = os.path.join(data_dir, setb)
setb_data_fnnames = os.listdir(setb_data_dir)

In [0]:
import time
from gwpy.timeseries import TimeSeries
epoch_noise = np.empty(3)
ifo_noise = []
images_noise = []

for i in range(0, 3, 1):
  epoch_noise[i] = int(glitch_data['GPStime'][i])
  ifo_noise.append(glitch_data['ifo'][i])
print(ifo_noise, epoch_noise)

for a in range(0, 3, 1):
  I = TimeSeries.fetch_open_data(ifo_noise[a], epoch_noise[a]-5, epoch_noise[a]-1).q_transform()
  images_noise.append(I)
  fig, ax = plt.subplots(1, 1, figsize = (10, 5))
  ax.imshow(I)
  ax.set_xscale('seconds')
  ax.set_yscale('log')
  ax.set_ylabel('Frequency [Hz]')
  ax.grid(True, axis='y', which='both')
  ax.colorbar(cmap='viridis', label='Normalized energy')
  ax.set_title('Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a]))
  plt.savefig(os.path.join(setb_data_dir,'Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a])))
  time.sleep(0.01)

In [0]:
import time
from gwpy.timeseries import TimeSeries
epoch_glitch= np.empty(3)
ifo_glitch = []
interval = np.empty(3)
images_glitch = []

for i in range(0, 3, 1):
  epoch_glitch[i] = int(glitch_data['GPStime'][i])
  ifo_glitch.append(glitch_data['ifo'][i])
  interval[i] = round((glitch_data['GPStime'][i] %1), 3)
print(ifo_glitch, epoch_glitch, interval)

for a in range(0, 3, 1):
  I = TimeSeries.fetch_open_data(ifo_glitch[a], epoch_glitch[a]-2, epoch_glitch[a]+2).q_transform(outseg = (epoch_glitch[a]+interval[a]-0.1 , epoch_glitch[a] +interval[a]+ 0.1))
  images_glitch.append(I)
  fig, ax = plt.subplots(1, 1, figsize = (10, 5))
  ax = plt.gca()
  ax.imshow(I)
  #ax.set_epoch(epoch_glitch[a])
  ax.set_xscale('seconds')
  ax.set_yscale('log')
  ax.set_ylabel('Frequency [Hz]')
  ax.grid(True, axis='y', which='both')
  ax.colorbar(cmap='viridis', label='Normalized energy', clim=[0, 40])
  ax.set_title('Q-Transform for Glitch at %s on GPS %d' %(ifo_glitch[a], epoch_glitch[a]))
  plt.savefig(os.path.join(seta_data_dir,'Q-Transform for Glitch at %s on GPS %d' %(ifo_noise[a], epoch_noise[a])))
  time.sleep(0.01)